This workbook runs the same code several times, and times the result
 - interpreted python
 - compiled cuda python 1 CPU core
 - compiled cuda python all CPU cores
 - compiled cuda python GPU, vectorize
 - compiled cuda python GPU, guvectorize, or njit
 current vector size uses 12 Gb in total
 
 The interpreted python is run on a smaller sample, in order to reduce run times
 the vectorise version can only process 1/10 of the data, as all needs to be stored on the GPU.
 My GPU only as 2Gb memory, so vectorize does not work for this sample data

Results on a i7 system with > 16 Gb RAM and 'Quadro P620' GPU with 2 Gb RAM

* interpreted python:        463.8 s
* complied CUDA, 1 CPU:       16.9 s
* complied CUDA, all CPUs:     3.6 s (all 8 CPU cores are used, but the 4 FPUs on the i7 are the bottleneck)
* vectorize GPU with smaller data set to fit in GPU memory: 4.3 s
* guvectorize GPU (scalable): 16.7 s
* njit GPU (scalable):        16.4 s


In [7]:
vec_size = 1000000000

In [8]:
import numpy as np
import resource
from timeit import default_timer as timer

def pow(a, b, c):
  for i in range(a.size):
     c[i] = a[i] ** b[i]

def main():
  vec_div=50
  vec_all = int(vec_size/vec_div)

  a = b = np.array(np.random.sample(vec_all), dtype=np.float32)
  c = np.zeros(vec_all, dtype=np.float32)

  start = timer()
  pow(a, b, c)
  duration = (timer() - start)*vec_div
  
  print(duration)
  usage= resource.getrusage(resource.RUSAGE_SELF)
  print(usage.ru_maxrss)

if __name__ == '__main__':
  main()

461.54617000001963
11966080


In [9]:
from numba import cuda
my_gpu=cuda.get_current_device()
gpu = cuda.get_current_device()
print("name = %s" % gpu.name)
print("maxThreadsPerBlock = %s" % str(gpu.MAX_THREADS_PER_BLOCK))
print("maxBlockDimX = %s" % str(gpu.MAX_BLOCK_DIM_X))
print("maxBlockDimY = %s" % str(gpu.MAX_BLOCK_DIM_Y))
print("maxBlockDimZ = %s" % str(gpu.MAX_BLOCK_DIM_Z))
print("maxGridDimX = %s" % str(gpu.MAX_GRID_DIM_X))
print("maxGridDimY = %s" % str(gpu.MAX_GRID_DIM_Y))
print("maxGridDimZ = %s" % str(gpu.MAX_GRID_DIM_Z))
print("maxSharedMemoryPerBlock = %s" % str(gpu.MAX_SHARED_MEMORY_PER_BLOCK))
print("asyncEngineCount = %s" % str(gpu.ASYNC_ENGINE_COUNT))
print("canMapHostMemory = %s" % str(gpu.CAN_MAP_HOST_MEMORY))
print("multiProcessorCount = %s" % str(gpu.MULTIPROCESSOR_COUNT))
print("warpSize = %s" % str(gpu.WARP_SIZE))
print("unifiedAddressing = %s" % str(gpu.UNIFIED_ADDRESSING))
print("pciBusID = %s" % str(gpu.PCI_BUS_ID))
print("pciDeviceID = %s" % str(gpu.PCI_DEVICE_ID))

name = b'Quadro P620'
maxThreadsPerBlock = 1024
maxBlockDimX = 1024
maxBlockDimY = 1024
maxBlockDimZ = 64
maxGridDimX = 2147483647
maxGridDimY = 65535
maxGridDimZ = 65535
maxSharedMemoryPerBlock = 49152
asyncEngineCount = 2
canMapHostMemory = 1
multiProcessorCount = 4
warpSize = 32
unifiedAddressing = 1
pciBusID = 1
pciDeviceID = 0


In [10]:
import numpy as np
from timeit import default_timer as timer
from numba import vectorize

@vectorize(['float32(float32, float32)'], target='parallel')
def powp(a, b):
  return a ** b

@vectorize(['float32(float32, float32)'], target='cpu')
def powc(a, b):
  return a ** b


def main():

  a = b = np.array(np.random.sample(vec_size), dtype=np.float32)
  c = np.zeros(vec_size, dtype=np.float32)

  start = timer()
  c = powp(a, b)
  duration = timer() - start

  print(duration)

  start = timer()
  c = powc(a, b)
  duration = timer() - start

  print(duration)
  usage= resource.getrusage(resource.RUSAGE_SELF)
  print(usage.ru_maxrss)

if __name__ == '__main__':
  main()

3.628375058000529
16.723290100000668
11967464


In [11]:
import numpy as np
from timeit import default_timer as timer
from numba import vectorize,guvectorize,jit,njit
from numba import cuda,float32

@vectorize(['float32(float32, float32)'], target='cuda')
def powv(a, b):
  return a ** b

def main():

  vec_div=10
  vec_all = int(vec_size/vec_div)

  a = b = np.array(np.random.sample(vec_all), dtype=np.float32)
  c = np.zeros(vec_all, dtype=np.float32)

  start = timer()
  c = powv(a, b)
  duration = (timer() - start)*vec_div

  print(duration)
  usage= resource.getrusage(resource.RUSAGE_SELF)
  print(usage.ru_maxrss)    

if __name__ == '__main__':
  main()

4.680918819994986
11967464


In [12]:
import numpy as np
from timeit import default_timer as timer
from numba import vectorize,guvectorize,jit,njit
from numba import cuda,float32

@guvectorize([(float32[:], float32[:], float32[:])], '(n),(n)->(n)')
def powg(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] ** y[i]

#@njit([(float32[:], float32[:], float32[:])],parallel = True,fastmath = True)
@njit([(float32[:], float32[:], float32[:])])
def pown(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] ** y[i]

def main():

  a = b = np.array(np.random.sample(vec_size), dtype=np.float32)
  c = np.zeros(vec_size, dtype=np.float32)

  start = timer()
  powg(a, b,c)
  duration = timer() - start
  print(duration)
  
  start = timer()
  pown(a, b,c)
  duration = timer() - start

  print(duration)
  usage= resource.getrusage(resource.RUSAGE_SELF)
  print(usage.ru_maxrss)    

if __name__ == '__main__':
  main()

16.76100386500002
16.356797776999883
11974956
